In [ ]:
import cv2
import numpy as np
import joblib
import tensorflow as tf
from tensorflow.keras.models import load_model
import os

# --- SETTINGS ---
IMAGE_SIZE = 48
MODEL_PATH = '/home/pai/hybrid_stress_model.h5'
ENCODER_PATH = '/home/pai/label_encoder.joblib'
CAPTURE_PATH = 'captured_image.jpg'

# --- 1. Capture from webcam ---
def capture_image():
    print("📸 Press SPACE to capture image or ESC to exit")
    cam = cv2.VideoCapture(0)

    while True:
        ret, frame = cam.read()
        if not ret:
            print("❌ Failed to grab frame")
            break
        cv2.imshow("Live Feed", frame)

        key = cv2.waitKey(1)
        if key % 256 == 27:  # ESC
            print("❌ Capture cancelled.")
            return None
        elif key % 256 == 32:  # SPACE
            cv2.imwrite(CAPTURE_PATH, frame)
            print(f"✅ Image saved to {CAPTURE_PATH}")
            break

    cam.release()
    cv2.destroyAllWindows()
    return CAPTURE_PATH

# --- 2. Preprocess the image ---
def preprocess_image(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=-1)  # (48, 48, 1)
    img = np.expand_dims(img, axis=0)   # (1, 48, 48, 1)
    return img

# --- 3. Load model & encoder, predict ---
def predict_stress(image):
    model = load_model(MODEL_PATH)
    le = joblib.load(ENCODER_PATH)
    prediction = model.predict(image)
    class_index = np.argmax(prediction)
    stress_label = le.inverse_transform([class_index])[0]
    return stress_label

# --- MAIN ---
def main():
    path = capture_image()
    if not path:
        return
    image = preprocess_image(path)
    result = predict_stress(image)
    print(f"\n🔍 Predicted Stress Level: **{result.upper()}**")

if __name__ == '__main__':
    main()
